    Loading the Dataset

In [1]:
import pandas as pd
df = pd.concat([pd.read_csv('Whole_dataset1.txt'),\
                pd.read_csv('Whole_dataset2.txt')\
,pd.read_csv('Whole_dataset3.txt'),pd.read_csv('Whole_dataset4.txt')])
import gc; gc.collect()
df.shape

(100480507, 4)

### Mapping the data
    Proper mapping must be done so that sparse matrix can be built

In [2]:
import numpy as np
df = df.sort_values(by='CustomerID')
gc.collect()
UID = np.array(df.CustomerID)
count = 0; flag=UID[0]
for i in range(UID.shape[0]):
    if(flag==UID[i]):
        UID[i]=count
    else:
        count += 1
        flag=UID[i]
        UID[i]=count
df['CustomerID'] = UID; del UID
df = df.sort_values(by='MovieID')
MID = np.array(df.MovieID)
count = 0; flag=MID[0]
for i in range(MID.shape[0]):
    if(flag==MID[i]):
        MID[i]=count
    else:
        count += 1
        flag=MID[i]
        MID[i]=count
df['MovieID'] = MID; del MID
df.MovieID.min(), df.MovieID.max(), df.CustomerID.min(), df.CustomerID.max()

(0, 17769, 0, 480188)

### Converting Date to timestamp

In [3]:
timestamp = np.array(df.Date)
tmp_time = np.empty(timestamp.shape[0])
for i in range(timestamp.shape[0]):
    t = timestamp[i].split('-')
    t = ''.join(t)
    tmp_time[i] = int(t)
del timestamp
df['Date'] = tmp_time
df.head(3)

,CustomerID,Score,Date,MovieID
351,283659,4,20040818.0,0
298,406494,5,20050829.0,0
410,258417,5,20050808.0,0


### Train Test split
    Setting the test data as 0.000001 in the sparse matrix

In [4]:
import numpy as np
df = df.sort_values(by='Date')
print('.')
UID = np.array(df.CustomerID)
MID = np.array(df.MovieID)
Score = np.array(df.Score)
l = int(df.shape[0]*0.7)
for i in range(l,df.shape[0]):
    Score[i]=0.000001
del df    
gc.collect()

.


339

### Creating scipy.CSR

In [5]:
from scipy.sparse import csr_matrix
SPM = csr_matrix((Score, (UID, MID)), shape=(480189,17770))
SPM

<480189x17770 sparse matrix of type '<class 'numpy.int64'>'
	with 100480507 stored elements in Compressed Sparse Row format>

In [6]:
import pickle
with open('Whole-data-sparse-matrix.pkl','wb') as fp:
    pickle.dump(SPM,fp)
fp.close(); del SPM